In [2]:
import tensorflow as tf
import os
import matplotlib.pyplot as plt

/Users/beomyongnho/.pyenv/versions/3.6.1/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [3]:
tf.reset_default_graph()

In [4]:
image_dir = './tfrecord_dataset/images_png'
label_name = './tfrecord_dataset/label_csv/label.csv'
image_names = [os.path.join(image_dir, n) for n in os.listdir(image_dir)]
image_names = sorted(image_names)

#print(os.listdir(image_dir))

img_name_queue = tf.train.string_input_producer(image_names, num_epochs=1, shuffle=False) 
#shuffle을 하지 않는 이유는 label과의 매칭을 위해... 아니면 label과 seed값을 동일하게 맞춰도 된다. 
label_name_queue = tf.train.string_input_producer([label_name], num_epochs=1, shuffle=False)

img_reader = tf.WholeFileReader()
text_reader = tf.TextLineReader()

key, value = img_reader.read(img_name_queue) #key는 파일명, value는 image의 binary data
txt_key, txt_value = text_reader.read(label_name_queue)

img_png = tf.image.decode_png(value)
img_png = tf.reduce_mean(img_png, axis=-1) # 3차원 배열을 2차원 배열로 변경
img_png = tf.reshape(img_png, [-1]) # img의 2차원 배열을 1차원으로 flatten

txt_csv = tf.decode_csv(txt_value, record_defaults=[[0]])

###
#with tf.Session() as sess:
#    print(sess.run(img_png))
### 이 상태로는 queue만 만들어지고 thread는 동작하지 않음 => 무한대기 상태

In [6]:
with tf.Session() as sess:
    sess.run(tf.local_variables_initializer())
    coord = tf.train.Coordinator() 
    thread = tf.train.start_queue_runners(sess, coord)
    
    face_train_dir = './cnn_dataset/face_train.tfrecord'
    face_test_dir = './cnn_dataset/face_test.tfrecord'
    
    train_writer = tf.python_io.TFRecordWriter(face_train_dir)
    test_writer = tf.python_io.TFRecordWriter(face_test_dir)
    
    for i in range(9999999):
        try: # 데이터 사이즈를 모를 땐 예외처리
            _img, _age = sess.run([img_png, txt_csv])

            example = tf.train.Example()
            example.features.feature['age'].int64_list.value.append(_age[0])
            example.features.feature['img'].int64_list.value.extend(_img.tolist())
         
            if i < 6000:
                train_writer.write(example.SerializeToString())
                if i%500 == 0:
                    print('{} train data written'.format(i))
            else:
                test_writer.write(example.SerializeToString())
                if i%500 == 0:
                    print('{} test data written'.format(i))
 

        except tf.errors.OutOfRangeError:
            print('end of record')
            break
            
    train_writer.close()
    test_writer.close()
    
    coord.request_stop()
    coord.join(thread)

0 train data written
500 train data written
1000 train data written
1500 train data written
2000 train data written
2500 train data written
3000 train data written
3500 train data written
4000 train data written
4500 train data written
5000 train data written
5500 train data written
6000 test data written
6500 test data written
7000 test data written
7500 test data written
end of record


In [ ]:
import tensorflow as tf
import os

image_dir = './cnn_dataset/dataset_img'
age_dir = './cnn_dataset/dataset_age/age.csv'
imgname_list = [os.path.join(image_dir, n) for n in os.listdir(image_dir)]

imgname_queue = tf.train.string_input_producer(imgname_list, num_epochs=1, shuffle=False, )
agename_queue= tf.train.string_input_producer([age_dir], shuffle=False)

img_reader = tf.WholeFileReader()
age_reader = tf.TextLineReader()

_, raw_img = img_reader.read(imgname_queue)
_, raw_age = age_reader.read(agename_queue)

decoded_img = tf.image.decode_png(raw_img)
decoded_age = tf.decode_csv(raw_age, [[0]])

reshaped_img = tf.reshape(decoded_img, [-1])

In [ ]:
with tf.Session() as sess:
    coord = tf.train.Coordinator()
    thread = tf.train.start_queue_runners(sess, coord)
    sess.run(tf.local_variables_initializer())

    face_train_dir = './cnn_dataset/face_train.tfrecord'
    face_test_dir = './cnn_dataset/face_test.tfrecord'
    
    train_writer = tf.python_io.TFRecordWriter(face_train_dir)
    test_writer = tf.python_io.TFRecordWriter(face_test_dir)
    
    for i in range(99999999):
        try:
            _age, _img = sess.run([decoded_age, reshaped_img])

            example = tf.train.Example()
            example.features.feature['age'].int64_list.value.append(_age[0])
            example.features.feature['img'].int64_list.value.extend(_img.tolist())

            if i < 6000:
                train_writer.write(example.SerializeToString())
                if i%500 == 0:
                    print('{} train data written'.format(i))
            else:
                test_writer.write(example.SerializeToString())
                if i%500 == 0:
                    print('{} test data written'.format(i))

        except tf.errors.OutOfRangeError:
            print('size of total dataset {}'.format(i))
            break
            
    train_writer.close()
    test_writer.close()
    
    coord.request_stop()
    coord.join(thread)